In [2]:
%run "Import.ipynb"

class DataAnalysis(DataOperation):
    """
    Führt eine Analyse und Validierung von Datensätzen durch. Die Hauptfunktionen sind folgende:
    1. Identifiziert die besten Passungen zwischen Trainingsdaten und idealer Funktionen mittels der Least-Square-Methode               
    2. Berechnet maximale Abweichungen und validiert Testdaten gegen diese Passungen. 
    3. Die Ergebnisse der Analyse und Validierung werden in strukturierten Datenformaten bereitgestellt und können für weitere Datenvisualisierungen und -analysen verwendet werden.

    Methoden:
        load_data_from_sql: Lädt die erforderlichen Datensätze aus der SQLite-DB.
        find_best_fits: Identifiziert die besten Passungen für die Trainingsdaten anhand der Least-Square-Methode.
        calculate_max_deviations: Berechnet die maximalen Abweichungen zwischen den Trainingsdaten und den idealen Funktionen.
        validate_test_data: Überprüft die Anwendbarkeit der identifizierten besten Passungen auf Testdaten und validiert diese.
        analyze_and_validate: Koordiniert den Gesamtprozess von Laden, Analysieren und Validieren der Daten und gibt die Ergebnisse         zurück.
    
    Attributes:
        train_df (DataFrame): Enthält die Trainingsdaten.
        ideal_df (DataFrame): Enthält die idealen Funktionen.
        test_df (DataFrame): Enthält die Testdaten.
        best_fits_df (DataFrame): Hält die besten Passungen zwischen Trainingsdaten und idealen Funktionen.
        max_deviations (Dict): Speichert die maximalen Abweichungen für jede beste Passung.
        valid_test_results_df (DataFrame): Enthält die validierten Testdaten und deren Abweichungen.
    """
    
    def load_data_from_sql(self):
        """
        Lädt Trainings-, ideale und Testdaten aus der SQLite-DB.

        Raises:
            Exception: Fehler beim Laden der Daten aus der DB.
        """
        try:
            self.train_df = pd.read_sql('SELECT * FROM training_data', self.conn)
            self.ideal_df = pd.read_sql('SELECT * FROM ideal_functions', self.conn)
            self.test_df = pd.read_sql('SELECT * FROM test_data', self.conn)
        except Exception as e:
            print(f"Fehler beim Laden der Daten aus der DB: {e}")
            raise

    def find_best_fits(self):
        """
        Findet die besten Passungen für die Trainingsdaten anhand der Least-Square Methode.

        Raises:
            Exception: Fehler beim Finden der besten Passungen.
        """
        best_fits = []
        try:
            for i in range(1, 5):  # Für y1 bis y4 in den Trainingsdaten
                best_fit, lowest_sum_sq_diff = None, np.inf
                for col in self.ideal_df.columns[1:]:  # Ignoriert x-Spalte
                    sq_diff = np.sum((self.train_df[f'y{i}'] - self.ideal_df[col]) ** 2)
                    if sq_diff < lowest_sum_sq_diff:
                        best_fit, lowest_sum_sq_diff = col, sq_diff
                best_fits.append((f'y{i}', best_fit, lowest_sum_sq_diff))
            self.best_fits_df = pd.DataFrame(best_fits, columns=["Trainingsdatensatz", "Beste Passung", "Summe quadratischer Abweichungen"])
        except Exception as e:
            print(f"Fehler beim Finden der besten Passungen: {e}")
            raise

    def calculate_max_deviations(self):
        """
        Berechnet die maximalen Abweichungen zwischen Trainingsdaten und den idealen Funktionen.

        Raises:
            Exception: Fehler bei der Berechnung der maximalen Abweichungen.
        """
        self.max_deviations = {}
        try:
            for train_y, best_fit, _ in self.best_fits_df.itertuples(index=False):
                max_deviation = np.max(np.abs(self.train_df[train_y] - self.ideal_df[best_fit]))
                self.max_deviations[best_fit] = max_deviation
        except Exception as e:
            print(f"Fehler bei der Berechnung der maximalen Abweichungen: {e}")
            raise          
            
    def validate_test_data(self):
        """
        Validiert die Testdaten gegen die besten Passungen.

        Raises:
            Exception: Fehler bei der Validierung der Testdaten.
        """
        valid_test_results = []
        max_allowed_deviation_factor = np.sqrt(2)
        try:
            for _, row in self.test_df.iterrows():
                x_value, y_test = row['x'], row['y']
                for ideal_col, max_dev in self.max_deviations.items():
                    y_ideal = np.interp(x_value, self.ideal_df['x'], self.ideal_df[ideal_col])
                    deviation = np.abs(y_test - y_ideal)
                    if deviation <= max_dev * max_allowed_deviation_factor:
                        valid_test_results.append((x_value, y_test, ideal_col, deviation))
            self.valid_test_results_df = pd.DataFrame(valid_test_results, columns=['x', 'y', 'Ideale Funktion', 'Abweichung'])
        except Exception as e:
            print(f"Fehler bei der Validierung der Testdaten: {e}")
            raise
            
    def analyze_and_validate(self):
        """
        Führt die komplette Analyse und Validierung durch und gibt die Ergebnisse zurück.

        Raises:
            Exception: Fehler während der Analyse und Validierung.

        Returns:
            best_fits_df (DataFrame), max_deviations (Dict.), valid_test_results_df (DataFrame)
        """
        try:
            self.load_data_from_sql()
            self.find_best_fits()
            self.calculate_max_deviations()
            self.validate_test_data()
            print("Analyse und Validierung erfolgreich abgeschlossen.")
            return self.best_fits_df, self.max_deviations, self.valid_test_results_df
        except Exception as e:
            print(f"Fehler während der Analyse und Validierung: {e}")
            raise

# Initialisierung und Ausführung
data_analysis = DataAnalysis('sqldatabase.sqlite')
data_analysis.connect_to_db()
best_fits_df, max_deviations, valid_test_results_df = data_analysis.analyze_and_validate()

DB-Verbindung erfolgreich hergestellt.
SQL-DB erstellt
Daten erfolgreich importiert zu training_data.
Daten erfolgreich importiert zu ideal_functions.
Daten erfolgreich importiert zu test_data.
SQL-Tabellen erstellt
DB-Verbindung erfolgreich hergestellt.
Analyse und Validierung erfolgreich abgeschlossen.
